# Tarea 05 - Para entregar

## Datos de _A practical guide to SVM classification_

Tres conjuntos de datos descritos y empleados en el documento _[A practical guide to SVM classification](http://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf),_ mencionado más arriba. La descripción de los datos  aparece en la tabla 1 de la página 2. Los datos mismos se pueden descargar en el enlace dado en el documento mismo, o bien el primero _(Astroparticle, `train` y `test`)_ y el tercero _(Vehicle, `train` y `test`)_ están también en la colección de datasets de la web [LIBSVM Data: Classification, Regression, and Multi-label](http://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/), bajo los nombres [`svmguide1`](http://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html#svmguide1) y [`svmguide3`](http://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html#svmguide3).

### (a) Preprocesado de los datos

El primer objetivo con estos datos es prepararlos para su tratamiento. Por alguna razón estos datos son más en bruto que lo que es usual: en primer lugar en el primer enlace no vienen en forma de una observación por línea de fichero, en el segundo sí. Además, cada valor de las variables predictoras viene en el siguiente formato:

 `[número de la variable (un entero), ":",valor en punto flotante]`,

sin espacios intercalados. 

Según la descripción del documento los datos `Vehicle` tienen 21 predictores, mientras que en el fichero de datos algunas observaciones tienen una variable predictora adicional con el número 22. Todos los valores que aparecen de esta variable 22 son iguales a 0, de modo que no aporta ninguna información, pero hay que tener en cuenta su presencia al leer el fichero: no todos los registros tienen la misma longitud. 

Por tanto, al menos para los datos `Bioinformatics` se requiere algun tipo de lectura a bajo nivel, ya sea en R con la función `scan()` o mediante algun otro programa. La eliminación de las etiquetas de las variables puede hacerse en un editor ascii o mediante lectura a bajo nivel.

In [1]:
astroparticle.test<-read.table("svmguide1.test.txt", header=TRUE)
astroparticle.train<-read.table("svmguide1.train.txt", header=TRUE)

astroparticle.test[,1]<-as.factor(astroparticle.test[,1])
astroparticle.train[,1]<-as.factor(astroparticle.train[,1])

Bioinformatics<-read.table("svmguide2.txt", header=TRUE)
Bioinformatics[,1]<-as.factor(Bioinformatics[,1])


vehicle.test<-read.table("svmguide3.test.txt", header=TRUE)
vehicle.test<-vehicle.test[-ncol(vehicle.test)]
vehicle.test[,1]<-as.factor(vehicle.test[,1])


vehicle.train<-read.table("svmguide3.train.txt", header=TRUE)
vehicle.train<-vehicle.train[-ncol(vehicle.train)]
vehicle.train[,1]<-as.factor(vehicle.train[,1])


El resultado de esta primera etapa debe ser:

Dos `data.frame`, `astroparticle.train` y `astroparticle.test`, con una variable respuesta llamada `y`, que puede ser un entero (valores 0/1) o bien un factor con dos niveles, y las variables predictoras, `x1`, `x2`,`x3`,`x4`.
   
Dos `data.frame`, `vehicle.train` y `vehicle.test`,  con una variable respuesta llamada `y`, que puede ser un entero (valores 0/1) o bien un factor con dos niveles, y variables predictoras `x01` a `x21`.

Un `data.frame`, `Bioinformatics`, con una variable respuesta llamada `y`, un factor con tres niveles, y variables predictoras `x01` a `x20`. 

En `Bioinformatics` no nos proporcionan un conjunto de test separado. Se puede separar una porción de datos con esta finalidad, como en prácticas anteriores, o bien emplear validación cruzada $k$-fold. También, por supuesto, los datos que vienen divididos en `train` y `test` pueden reunirse para aplicar cualquier procedimiento de validación cruzada, incluyendo _otra_ subdivisión `train` y `test`. La utilidad de la subdivisión que viene dada es poder reproducir resultados publicados.

In [2]:
str(astroparticle.train)
str(astroparticle.test)

'data.frame':	3089 obs. of  5 variables:
 $ y : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ x1: num  26.2 57.1 17.3 21.8 91.3 ...
 $ x2: num  58.9 221.4 173.4 125 293.6 ...
 $ x3: num  -0.1895 0.0861 -0.1298 0.1539 0.1424 ...
 $ x4: num  125 123 125 153 161 ...
'data.frame':	4000 obs. of  5 variables:
 $ y : Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 1 1 ...
 $ x1: num  4.24 4.24 1.66 2.8 2.84 ...
 $ x2: num  22 22 80.7 22.4 30.2 ...
 $ x3: num  -0.35 -0.35 0.213 0.341 0.386 ...
 $ x4: num  97.5 97.5 68.7 81 94.9 ...


In [3]:
Bioinformatics<-Bioinformatics[sample(nrow(Bioinformatics)),]
indexesTrain <- c(1:(nrow(Bioinformatics)/2))
indexesTest <- c((nrow(Bioinformatics)/2+1):nrow(Bioinformatics))

Bioinformatics.train<-Bioinformatics[indexesTrain,]
Bioinformatics.test<-Bioinformatics[indexesTest,]

str(Bioinformatics.train)
str(Bioinformatics.test)

'data.frame':	195 obs. of  21 variables:
 $ y  : Factor w/ 3 levels "1","2","3": 1 2 1 2 2 1 1 1 1 2 ...
 $ x1 : num  0.0599 0.0522 0.0813 0.064 0.0414 ...
 $ x2 : num  0.1058 0.1082 0.0517 0.0853 0.0602 ...
 $ x3 : num  0.0519 0.0485 0.0665 0.0521 0.0451 ...
 $ x4 : num  0.02 0.0261 0.0345 0.0213 0.0188 ...
 $ x5 : num  0.0299 0.0448 0.0296 0.045 0.0301 ...
 $ x6 : num  0.02395 0.01493 0.00246 0.02133 0.03008 ...
 $ x7 : num  0.0439 0.0299 0.032 0.0355 0.0451 ...
 $ x8 : num  0.0818 0.0522 0.1084 0.0853 0.0789 ...
 $ x9 : num  0.0818 0.1045 0.1281 0.1256 0.1241 ...
 $ x10: num  0.0399 0.0522 0.0271 0.0308 0.0451 ...
 $ x11: num  0.0599 0.0522 0.0714 0.0498 0.0301 ...
 $ x12: num  0.0499 0.0448 0.0665 0.0711 0.0526 ...
 $ x13: num  0.00599 0.01119 0.01724 0.00474 0.03008 ...
 $ x14: num  0.02994 0.02612 0.02217 0.00474 0.02256 ...
 $ x15: num  0.0778 0.0784 0.0369 0.0284 0.0677 ...
 $ x16: num  0.0499 0.041 0.0468 0.0569 0.0451 ...
 $ x17: num  0.0319 0.0634 0.0148 0.0474 0.0827 ...
 $

In [4]:
str(vehicle.train)
str(vehicle.test)

'data.frame':	1243 obs. of  22 variables:
 $ y  : Factor w/ 2 levels "-1","1": 1 1 1 1 1 1 1 1 1 1 ...
 $ x1 : num  0.0643 0.0414 0.0357 0.0343 0.1129 ...
 $ x2 : num  -0.000885 -0.003728 -0.006948 -0.00541 -0.009484 ...
 $ x3 : num  7.17e-05 1.01e-03 3.96e-03 2.53e-03 2.34e-03 ...
 $ x4 : num  -0.2638 -0.0196 -0.2902 -0.0221 -0.1141 ...
 $ x5 : num  0.00389 0.00184 0.05911 0.00365 0.01002 ...
 $ x6 : num  -2.93e-05 -4.56e-05 -3.13e-03 -1.57e-04 -2.28e-04 ...
 $ x7 : num  0.108449 0.000989 0.233714 0.001466 0.011546 ...
 $ x8 : num  -5.75e-04 -3.26e-05 -1.69e-02 -8.50e-05 -3.58e-04 ...
 $ x9 : num  1.12e-05 2.07e-06 2.32e-03 9.34e-06 2.10e-05 ...
 $ x10: int  1 1 1 1 1 1 1 1 1 1 ...
 $ x11: num  0 0.803 0 0.781 0.843 ...
 $ x12: num  0.798 0 0.738 0 0.853 ...
 $ x13: num  0.5099 0.0968 0.5073 0.0973 0.2981 ...
 $ x14: num  0 0.1 0 0.1 0.1 0.1 0 0 0.1 0.2 ...
 $ x15: num  0.1 0.1 0.1 0 0.3 0.1 0 0.1 0 0 ...
 $ x16: num  0.533 0.567 0.538 0.55 0.558 ...
 $ x17: num  0.15 0.323 0.321 0.37

## (b) Clasificación de los tres datasets

La siguiente etapa va a ser la construcción y evaluación de procedimientos de clasificación para estos tres conjuntos de datos, empleando:

1. Regresión logística, para 1 y 3 (que tienen dos clases),
2. `lda`, para los tres,
3. _Bagging,_ _Random Forest,_ _AdaBoost,_ para los tres,
4. `svm`, para los tres,

Para los dos primeros procedimientos, que no tienen parámetros ajustables, obtened la matriz de confusión y una estimación de la probabilidad de clasificación errónea. Sin embargo, podéis hacer selección de predictores.

En las máquinas de vectores soporte, seguid las explicaciones del documento-guía  _[A practical guide to SVM classification](http://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf),_ ajustando los parámetros `C` y `nu`. Podeis hacer la búsqueda sistemática mediante la función `tune()`. En cualquier caso, intentad llegar a una precisión próxima a la que aparece en la última columna de la Tabla 1 del documento.

In [17]:
require(MASS)
require(e1071)
require(rpart)
require(adabag)
require(randomForest)
require(ada)

### 1. Regresion logica

In [6]:
#astroparticle.train astroparticle.test

#Creacion del modelo para la logistic regression de astroparticle
astroparticle.glm <- glm(y~.,family=binomial,data=astroparticle.train)

#prediccion Astroparticle, los resultados superiores a 0.5 seran de la clase 1 y los demas de la clase 0
astroparticle.pred <- predict(astroparticle.glm,newdata=astroparticle.test,type='response')
astroparticle.pred <- ifelse(astroparticle.pred > 0.5,1,0)

#error Astroparticle y matriz de confusion
astroparticle.error <- mean(astroparticle.pred != astroparticle.test$y)
C<-table("True"=astroparticle.test$y,"Predicted"=astroparticle.pred)
C
print(paste('Accuracy astroparticle',1-astroparticle.error))

Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"

    Predicted
True    0    1
   0 1908   92
   1   88 1912

[1] "Accuracy astroparticle 0.955"


In [7]:
#vehicle.train vehicle.test

#Creacion del modelo para la logistic regression de vehicle
vehicle.glm <- glm(y~.,family=binomial,data=vehicle.train)

#prediccion Vehicle, los resultados superiores a 0.5 seran de la clase 1 y los demas de la clase 0
vehicle.pred <- predict(vehicle.glm,newdata=vehicle.test,type='response')
vehicle.pred <- ifelse(vehicle.pred > 0.5,1,0)

#error Vehicle y matriz de confusion
vehicle.error <- mean(vehicle.pred != vehicle.test$y)
C<-table("True"=vehicle.test$y,"Predicted"=vehicle.pred)
C
print(paste('Accuracy vehicle',1-vehicle.error))

    Predicted
True  0  1
   1 16 25

[1] "Accuracy vehicle 0.609756097560976"


### 2. LDA

In [8]:
#astroparticle.train astroparticle.test

#modelo lda de astroparticle y prediccion del test
astroparticle.lda <- lda(formula = y~., data = astroparticle.train)
astroparticle.lda.pred <- predict(astroparticle.lda, newdata=astroparticle.test)
#matriz de confusion de las predicciones
C<-table("True"=astroparticle.test$y,"Predicted"=astroparticle.lda.pred$class)
C
#error de la prediccion de clasificacion
astroparticle.lda.error <- mean(astroparticle.lda.pred$class != astroparticle.test$y)
print(paste('Accuracy astroparticle',1-astroparticle.lda.error))

    Predicted
True    0    1
   0 1717  283
   1  157 1843

[1] "Accuracy astroparticle 0.89"


In [9]:
#Bioinformatics.train Bioinformatics.test

#modelo lda de Bioinformatics y prediccion del test
Bioinformatics.lda <- lda(formula = y~., data = Bioinformatics.train)
Bioinformatics.lda.pred <- predict(Bioinformatics.lda, newdata=Bioinformatics.test)
#matriz de confusion de las predicciones
C<-table("True"=Bioinformatics.test$y,"Predicted"=Bioinformatics.lda.pred$class)
C
#error de la prediccion de clasificacion
Bioinformatics.lda.error <- mean(Bioinformatics.lda.pred$class != Bioinformatics.test$y)
print(paste('Accuracy Bioinformatics',1-Bioinformatics.lda.error))

    Predicted
True   1   2   3
   1 101   7   0
   2   7  42   8
   3   7   6  17

[1] "Accuracy Bioinformatics 0.82051282051282"


In [10]:
#vehicle.train vehicle.test

#modelo lda de Bioinformatics y prediccion del test
vehicle.lda <- lda(formula = y~., data = vehicle.train)
vehicle.lda.pred <- predict(vehicle.lda, newdata=vehicle.test)
#matriz de confusion de las predicciones
C<-table("True"=vehicle.test$y,"Predicted"=vehicle.lda.pred$class)
C
#error de la prediccion de clasificacion
vehicle.lda.error <- C[1]/sum(C)
print(paste('Accuracy vehicle',1-vehicle.lda.error))

    Predicted
True -1  1
   1 16 25

[1] "Accuracy vehicle 0.609756097560976"


### 3. Bagging, Random Forest, AdaBoost 

Como aplicar cada uno es trivial, aplico uno a cada set de datos:
- Bagging en _'astroparticles'_
- Random Forest en _'Bioinformatics'_
- AdaBoost en _'vehicle'_

In [11]:
#astroparticle.train astroparticle.test

#Modelo Bagging de astroparticle con 10 remuestras bootstrap y prediccion del test
astroparticle.bagging<-bagging(y~.,data=astroparticle.train,mfinal=10)
astroparticle.bagging.pred<-predict.bagging(astroparticle.bagging,newdata=astroparticle.test)
#matriz de confusion de las predicciones
C<-astroparticle.bagging.pred$confusion
C
#error de la prediccion de clasificacion
astroparticle.bagging.error <- mean(astroparticle.bagging.pred$class != astroparticle.test$y)
print(paste('Accuracy astroparticle',1-astroparticle.bagging.error))

               Observed Class
Predicted Class    0    1
              0 1921   57
              1   79 1943

[1] "Accuracy astroparticle 0.966"


In [12]:
#Bioinformatics.train Bioinformatics.test

#Modelo Random Forest de Bioinformatics con 4 variables candidatas en cada muestra y prediccion del test
Bioinformatics.RF<-randomForest(y~.,data=Bioinformatics.train, mtry=4)
Bioinformatics.RF.pred<-predict(Bioinformatics.RF, newdata=Bioinformatics.test)
#matriz de confusion de las predicciones
C<-table("True"=Bioinformatics.test$y,"Pred"=Bioinformatics.RF.pred)
C
#error de la prediccion de clasificacion
Bioinformatics.RF.error <- mean(Bioinformatics.RF.pred != Bioinformatics.test$y)
print(paste('Accuracy Bioinformatics',1-Bioinformatics.RF.error))

    Pred
True  1  2  3
   1 99  9  0
   2 14 43  0
   3  9 12  9

[1] "Accuracy Bioinformatics 0.774358974358974"


In [13]:
#vehicle.train vehicle.test

#Modelo AdaBoost de vehicle y prediccion del test
vehicle.ada <- ada(y~., data=vehicle.train)
vehicle.ada.pred <- predict(vehicle.ada, vehicle.test)
#matriz de confusion de las predicciones
C<-table("True"=vehicle.test$y,"Pred"=vehicle.ada.pred)
C
#error de la prediccion de clasificacion
vehicle.ada.error <- C[1]/sum(C)
print(paste('Accuracy vehicle',1-vehicle.ada.error))

    Pred
True -1  1
   1  3 38

[1] "Accuracy vehicle 0.926829268292683"


### 4. SVM 

In [14]:
#astroparticle.train astroparticle.test

#Mejor modelo SVM de astroparticle obtenido por la funcion 'tune' y prediccion del test
astroparticle.svm <- tune.svm(y~.,data=astroparticle.train)$best.model
astroparticle.svm.pred <- predict(astroparticle.svm, astroparticle.test)
#matriz de confusion de las predicciones
C<-table("True"=astroparticle.test$y,"Pred"=astroparticle.svm.pred)
C
#error de la prediccion de clasificacion
astroparticle.svm.error <- mean(astroparticle.svm.pred != astroparticle.test$y)
print(paste('Accuracy astroparticle',1-astroparticle.svm.error))

    Pred
True    0    1
   0 1928   72
   1   56 1944

[1] "Accuracy astroparticle 0.968"


In [15]:
#Bioinformatics.train Bioinformatics.test

#Mejor modelo SVM de Bioinformatics obtenido por la funcion 'tune' y prediccion del test
Bioinformatics.svm <- tune.svm(y~.,data=Bioinformatics.train)$best.model
Bioinformatics.svm.pred <- predict(Bioinformatics.svm, Bioinformatics.test)
#matriz de confusion de las predicciones
C<-table("True"=Bioinformatics.test$y,"Pred"=Bioinformatics.svm.pred)
C
#error de la prediccion de clasificacion
Bioinformatics.svm.error <- mean(Bioinformatics.svm.pred != Bioinformatics.test$y)
print(paste('Accuracy Bioinformatics',1-Bioinformatics.svm.error))

    Pred
True   1   2   3
   1 103   5   0
   2   9  45   3
   3   7  10  13

[1] "Accuracy Bioinformatics 0.825641025641026"


In [16]:
#vehicle.train vehicle.test

#Mejor modelo SVM de vehicle obtenido por la funcion 'tune' y prediccion del test
vehicle.svm <- tune.svm(y~.,data=vehicle.train)$best.model
vehicle.svm.pred <- predict(vehicle.svm, vehicle.test)
#matriz de confusion de las predicciones
C<-table("True"=vehicle.test$y,"Pred"=vehicle.svm.pred)
C
#error de la prediccion de clasificacion
vehicle.svm.error <- C[1]/sum(C)
print(paste('Accuracy vehicle',1-vehicle.svm.error))

    Pred
True -1  1
   1 14 27

[1] "Accuracy vehicle 0.658536585365854"


O no entiendo como funciona la funcion "tune.svm" o los resultados usan unos parametros muy concretos en el modelo del documento.

Los resultados de _'astroparticle'_ cuadran a la perfeccion con los esperados, _'Bioinformatics'_ estan cerca, pero _'vehicle'_ esta muy lejos de lo esperado.

Al final, todas las opciones probadas funcionan muy bien con el primer set (entorno 95% de acierto),
el segundo sigue siendo bueno (entorno al 80% en todos los modelos), pero el tercero da malos resultados en logistic, lda y svm 
(este ultimo seria facilmente ajustable mediante los diversos parametros como el margen de error _'C'_) pero da una precision magnifica con adaBoost.

La mayoria de los modelos estan hechos con los parametros por defecto de la funcion, seria posible mejorar los resultados del segundo y tercer dataset mediante el ajuste de algunos parametros.